# T5-small 를 활용하여 Translation 구현하기 (Small Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 T5모델을 이용하여 아주 간단한 번역 문제를 풀어보겠습니다.

활용 데이터는 human-machine date 데이터로서 아주 간단한 seqeucne-to-sequence 문제입니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(훈련소요 시간 : 1시간, 설명 : 20분)

In [1]:
# performance metric
!pip install -q sacrebleu


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from IPython.display import Markdown, display

In [3]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Date Dataset

Dataset 은 이미 만들어 두었고, 이를 가져온다고 가정하겠습니다.

In [4]:
# 1) 원본 링크의 dl=0 → dl=1 로 변경
!wget -O human_machine_date_dataset.zip "https://www.dropbox.com/scl/fi/xar46xcg21xo1efzpjk7s/human_machine_date_dataset.zip?rlkey=b5xunuusad1z0h1yq1z17nbyx&st=fwsuz9bi&dl=1"


--2025-07-17 11:57:48--  https://www.dropbox.com/scl/fi/xar46xcg21xo1efzpjk7s/human_machine_date_dataset.zip?rlkey=b5xunuusad1z0h1yq1z17nbyx&st=fwsuz9bi&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.11.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.11.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb16d9de2e83931a9831bcca252.dl.dropboxusercontent.com/cd/0/inline/CtofPLhsmXBXnFOUJGlIbrt38xF3w-ZLBq5AeuFsafm4DB5VbNoJkM7s2h0krWDNTfUGmDuliUJBjQXtTZXhJGhPa7BTGbKt5amoFsvywDTmie1L1bVzBwNNtEIZ1pWXmVs/file?dl=1# [following]
--2025-07-17 11:57:49--  https://ucb16d9de2e83931a9831bcca252.dl.dropboxusercontent.com/cd/0/inline/CtofPLhsmXBXnFOUJGlIbrt38xF3w-ZLBq5AeuFsafm4DB5VbNoJkM7s2h0krWDNTfUGmDuliUJBjQXtTZXhJGhPa7BTGbKt5amoFsvywDTmie1L1bVzBwNNtEIZ1pWXmVs/file?dl=1
Resolving ucb16d9de2e83931a9831bcca252.dl.dropboxusercontent.com (ucb16d9de2e83931a9831bcca252.dl.dropboxusercontent.com)... 162.125.11.15, 

In [5]:
!unzip human_machine_date_dataset.zip

Archive:  human_machine_date_dataset.zip
   creating: human_machine_date_dataset/.ipynb_checkpoints/
 extracting: human_machine_date_dataset/.ipynb_checkpoints/dataset_dict-checkpoint.json  
 extracting: human_machine_date_dataset/dataset_dict.json  
   creating: human_machine_date_dataset/train/
  inflating: human_machine_date_dataset/train/cache-113ce0f9cbf9302b.arrow  
  inflating: human_machine_date_dataset/train/data-00000-of-00001.arrow  
  inflating: human_machine_date_dataset/train/dataset_info.json  
  inflating: human_machine_date_dataset/train/state.json  
   creating: human_machine_date_dataset/validation/
  inflating: human_machine_date_dataset/validation/cache-924eedb5e7ecb100.arrow  
  inflating: human_machine_date_dataset/validation/data-00000-of-00001.arrow  
  inflating: human_machine_date_dataset/validation/dataset_info.json  
  inflating: human_machine_date_dataset/validation/state.json  


In [6]:
from datasets import load_from_disk

data_directory = './human_machine_date_dataset'
dataset = load_from_disk(data_directory)

print("Loaded dataset:", dataset)

Loaded dataset: DatasetDict({
    train: Dataset({
        features: ['human', 'machine'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['human', 'machine'],
        num_rows: 1000
    })
})


In [7]:
dataset['train']['human'][:10]

['12, Sep 2010',
 'DECEMBER 14 1992',
 '8 JUN 1999',
 '11.03.00',
 '6/16/79',
 '1 FEBRUARY, 1988',
 '19 jun 1991',
 '12 january, 1981',
 '30 Sep 2015',
 '8 Jul, 1996']

In [8]:
dataset['train']['machine'][:10]

['2010-09-12',
 '1992-12-14',
 '1999-06-08',
 '2000-03-11',
 '1979-06-16',
 '1988-02-01',
 '1991-06-19',
 '1981-01-12',
 '2015-09-30',
 '1996-07-08']

## Data 전처리 하기

### Model 설정하기

In [9]:
MODEL_ID = "t5-small"

In [10]:
import transformers
from transformers import AutoTokenizer    
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# fast version 인지 아닌지 확인
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [11]:
max_token_length = 64

In [12]:
#print(tokenizer.vocab)

In [13]:
s = dataset["train"]['human'][0]
t = dataset["train"]['machine'][0]

s_output = tokenizer(s)
with tokenizer.as_target_tokenizer():
    t_output = tokenizer(t)

print( s_output['input_ids']  )
print( t_output['input_ids'] )
print("-"*50)
print( tokenizer.decode( s_output['input_ids'] ))
print( tokenizer.decode( t_output['input_ids'] ))

[10440, 12219, 2735, 1]
[2735, 18, 4198, 5947, 1]
--------------------------------------------------
12, Sep 2010</s>
2010-09-12</s>


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


### Prefix 넣어주기
T5 original paper에서는 모든 Task 에 task prefix를 넣도록 하는 방법을 제안합니다. 즉, 영어에서 한국어로 번역한다면 "Translate English to Korean : "이라는 prefix term 을 입력의 가장 앞에 넣어주는 방식입니다. 

그 이후에 나온 T5의 구현체마다 미미하게 다른 방식을 씁니다. 만약 Downstream task가 '번역'만 다룬다면 굳이 필요 없을 수도 있습니다. 

본 실습에서는 Prefix도 넣어주는 방법으로 구현해봅니다.

In [14]:
max_token_length = 64

ENABLE_PREFIX = True

source_lang = "human"
target_lang = "machine"

if ENABLE_PREFIX:
    prefix = f"translate {source_lang} to {target_lang}: "
else:
    prefix = ""
    
source_lang = source_lang.lower()
target_lang = target_lang.lower()
    
def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    
    tokenized_examples = tokenizer(
                            inputs,
                            text_target=targets, 
                            max_length=max_token_length, 
                            truncation=True
                         )
    return tokenized_examples

### tokenizer test해보기

In [15]:
print(preprocess_function(dataset['train'][:2]).keys())
out = preprocess_function(dataset['train'][:2])
print(out)
print("-"*50)
print("Input Tokens  : ", out['input_ids'][0])
print("Input IDs     : ", tokenizer.decode(out['input_ids'][0]) )
print("-"*50)
print("Output Tokens : ", out['labels'][0])
print("Output IDs    : ", tokenizer.decode(out['labels'][0]) )

KeysView({'input_ids': [[13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1], [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]})
{'input_ids': [[13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1], [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[2735, 18, 4198, 5947, 1], [9047, 5947, 11590, 1]]}
--------------------------------------------------
Input Tokens  :  [13959, 936, 12, 1437, 10, 10440, 12219, 2735, 1]
Input IDs     :  translate human to machine: 12, Sep 2010</s>
--------------------------------------------------
Output Tokens :  [2735, 18, 4198, 5947, 1]
Output IDs    :  2010-09-12</s>


### Dataset 전체에 걸쳐 전처리하기

## 만약 에러가 난다면 아래것을 활성화 하세요.

In [18]:
#!chmod -R 777 /workspace/8_aaa/human_machine_date_dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
tokenized_datasets = dataset.map(
                        preprocess_function, 
                        batched=True,
                        remove_columns=dataset["train"].column_names,
                     )

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [21]:
train_dataset = tokenized_datasets["train"]
valid_dataset = tokenized_datasets["validation"]

## T5 준비 

In [22]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

### Model Structure 확인

### Model Inference 수행해보기

In [23]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

번역에 맞는 몇 가지 입출력 전처리를 추가합니다.
1. Batch 내 입력, 출력의 최대 길이 조정
2. Input뿐만 아니라 Output레이블에도 Padding을 추가해줍니다.
3. Decoder input 에 start symbol에 해당하는 pad를 넣어줍니다.

In [24]:
sample_batch = [tokenized_datasets["train"][i] for i in range(1, 3)]

In [25]:
print( sample_batch[0] )
print( sample_batch[1] )

{'input_ids': [13959, 936, 12, 1437, 10, 3396, 254, 6037, 12920, 968, 9047, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [9047, 5947, 11590, 1]}
{'input_ids': [13959, 936, 12, 1437, 10, 505, 446, 7443, 5247, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [5247, 18, 5176, 18, 4018, 1]}


In [26]:
batch = data_collator(sample_batch)

In [27]:
batch.keys()

KeysView({'input_ids': tensor([[13959,   936,    12,  1437,    10,  3396,   254,  6037, 12920,   968,
          9047,     1],
        [13959,   936,    12,  1437,    10,   505,   446,  7443,  5247,     1,
             0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]]), 'labels': tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]]), 'decoder_input_ids': tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])})

In [28]:
print( batch['input_ids'] )

tensor([[13959,   936,    12,  1437,    10,  3396,   254,  6037, 12920,   968,
          9047,     1],
        [13959,   936,    12,  1437,    10,   505,   446,  7443,  5247,     1,
             0,     0]])


In [29]:
print( batch['attention_mask'] )

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [30]:
print( batch['labels'] )

tensor([[ 9047,  5947, 11590,     1,  -100,  -100],
        [ 5247,    18,  5176,    18,  4018,     1]])


In [31]:
print( batch['decoder_input_ids'] )

tensor([[    0,  9047,  5947, 11590,     1,     0],
        [    0,  5247,    18,  5176,    18,  4018]])


collator로 인해 2가지가 새로 변한 것을 알 수 있습니다.
- decoder_input_ids는 앞에 0 즉, pad symbol 이 추가되었습니다.
- labels 와 decoder_input_ids 를 살펴보면 하나씩 오른쪽으로 shift되었음을 확인할 수 있습니다.

In [32]:
#batch.to('cuda:0')
outputs = model(**batch)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [33]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [34]:
print(outputs.loss)
print(outputs.logits)

tensor(5.0773, grad_fn=<NllLossBackward0>)
tensor([[[-13.7437,  -8.1035, -12.8939,  ..., -41.6878, -41.7943, -41.7333],
         [-18.4979,  -1.3570,  -8.1574,  ..., -39.6779, -39.7441, -39.7231],
         [-14.5965,  -4.8751,  -5.9782,  ..., -37.9275, -37.9034, -37.8781],
         [-17.0803,  -2.6475,  -7.8406,  ..., -38.9906, -38.9632, -38.9547],
         [-25.2408,  -1.1384,  -7.6673,  ..., -39.6415, -39.6729, -39.7197],
         [-10.7400,  -5.1902, -10.7453,  ..., -37.6124, -37.7045, -37.7121]],

        [[-13.1328,  -8.7611, -12.9354,  ..., -39.8983, -39.9784, -39.9678],
         [-18.0095,  -1.3041,  -8.3104,  ..., -38.8964, -38.9699, -38.9826],
         [-16.1568,  -4.6274,  -5.3082,  ..., -34.2653, -34.2488, -34.2389],
         [-11.9368,  -1.9935,  -6.1218,  ..., -37.8455, -37.8817, -37.8955],
         [-14.0619,  -3.7020,  -5.3991,  ..., -34.2775, -34.2884, -34.3163],
         [-13.7334,  -1.7026,  -7.4004,  ..., -37.7079, -37.7560, -37.7729]]],
       grad_fn=<UnsafeViewBac

### 예측 결과를 Symbol 로 바꾸기

In [35]:
import torch
best_output = torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() 
tokenizer.convert_ids_to_tokens( best_output )

['▁Mensch', '</s>', '-', '</s>', '</s>', '▁Mensch']

어떻게든 답은 나오지만, 제대로 된 결과가 아님을 알 수 있습니다. 훈련을 통해 제대로 된 결과를 얻어보죠.

## Fine-Tuning 수행하기

### Evaluation Metric 

In [37]:
import evaluate
metric = evaluate.load("sacrebleu")

### SacreBLEU
SacreBLEU는 자연어 처리(NLP) 분야에서 기계 번역의 품질을 평가하기 위해 널리 사용되는 도구입니다. BLEU(BiLingual Evaluation Understudy) 점수는 기계 번역의 출력과 사람이 번역한 참조 번역 사이의 유사도를 측정하는 방법 중 하나로, SacreBLEU는 이 BLEU 점수를 보다 일관되고 신뢰할 수 있는 방식으로 계산합니다.

In [38]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result


위의 compute_metrics 는 [Transformer 번역 예제](https://huggingface.co/course/chapter7/4?fw=pt#metrics)에서 가져왔습니다.

### Huggingface 의 Trainer 활용하여 훈련하기

In [40]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    f"{MODEL_ID}-finetuned-human-to-machine",
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=2,             # 훈련 에폭 수
    per_device_train_batch_size=1024, # 디바이스 당 훈련 배치 크기
    
    ## ---- Learning Rate ----- ##
    #warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율
    learning_rate=2e-5,
    
    ## - Logging & Checkpoint - ##
    logging_steps=10,                # 로그를 기록할 스텝 간격
    save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
    
    ## ---- Evaluations ----- ##
    predict_with_generate=True,
    eval_strategy="epoch",
)


### Trainer 설정하기

In [41]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    
    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=valid_dataset,   # < test
    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_9821/4111572105.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [42]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [43]:
#torch.cuda.empty_cache()

In [44]:
trainer.train()

TrainOutput(global_step=978, training_loss=0.3698201069802594, metrics={'train_runtime': 378.8289, 'train_samples_per_second': 2639.714, 'train_steps_per_second': 2.582, 'total_flos': 4916925353164800.0, 'train_loss': 0.3698201069802594, 'epoch': 2.0})

In [45]:
trainer.save_model("./results")

## 훈련된 모델 Inference

In [46]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [47]:
input_text = ['translate human to machine: '+x for x in dataset['validation']['human'][:2] ]

In [48]:
input_text

['translate human to machine: 2/27/76',
 'translate human to machine: November 14, 1999']

In [49]:
inputs = tokenizer(input_text, 
                   return_tensors="pt", 
                   padding=True, 
                   max_length=max_token_length)

In [50]:
machines = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

machines.shape

torch.Size([2, 7])

In [51]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(m)) for m in machines
]

['<pad> 1976-02-27</s>', '<pad> 1999-11-14</s></s></s>']